In [15]:
#importing packages

import pandas as pd

import nltk
import spacy

from nltk.tokenize import word_tokenize
from spacy.tokens import Token

import re
import stopwordsiso as stopwords

In [16]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\furba\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [17]:
nlp = spacy.load('pl_core_news_sm')

In [18]:
#setting table display options

#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [19]:
articles_1 = pd.read_json('articledata.json') #reading article data
articles_1['text'] = articles_1['text'].apply(
    lambda x: str(x).replace('\\n', ' ') 
    if isinstance(x, (dict, list, str)) 
    else str(x)) #replacing \\n with space for dictionary, list or string and changing other datatypes to strings

In [20]:
articles_2 = pd.read_json('articledata_2.json') #reading other article data
articles_2['text'] = articles_2['text'].apply(
    lambda x: str(x).replace('\\n', ' ') 
    if isinstance(x, (dict, list, str)) 
    else str(x)) #replacing \\n with space for dictionary, list or string and changing other datatypes to strings

In [21]:
articles = pd.concat([articles_1, articles_2], axis=0) #concatanaiting both article data
articles = articles.drop_duplicates() #removing duplicates
articles = articles.sort_values(by='date', ascending=False) #sorting articles by date
articles.head()

,url,date,text
1,https://www.bankier.pl/wiadomosc/To-pietro-przyciaga-WIG20-jak-magnes-Portfelowe-przetasowania-na-GPW-8804616.html?utm_source=RSS&utm_medium=RSS&utm_campaign=Wiadomosci,2024-08-30 17:29:00,"['Ostatnia sesja miesiąca potwierdziła silne przywiązanie WIG20 do poziomu w okolicach 2400 pkt. W efekcie od połowy sierpnia obserwujemy trend boczny, z którego coraz trudniej się wyrwać. Portfelowe przetasowania zdecydowały o wyniku sesji i ogromnych obrotach.']"
0,https://www.bankier.pl/wiadomosc/Wyniki-goracych-spolek-i-parzacy-deficyt-budzetu-panstwa-8804278.html?utm_source=RSS&utm_medium=RSS&utm_campaign=Wiadomosci,2024-08-30 13:30:00,"['Ostatni tydzień wakacji na rynkach finansowych był podporządkowany wynikom Nvidii. Inwestorzy na krajowym parkiecie dostali natomiast raport CD Projektu. Kurs największego polskiego studia gier jest w tym roku najlepszy w portfelu spółek z WIG20, ale jego cena akcji zdaje się być coraz bardziej wymagająca. Nie ominiemy także podsumowania rywalizacji w grze giełdowej „Wakacje na giełdzie”.']"
2,https://www.bankier.pl/wiadomosc/Widoczna-slabosc-WIG20-Budzetowe-sesja-na-GPW-8803440.html?utm_source=RSS&utm_medium=RSS&utm_campaign=Wiadomosci,2024-08-28 17:03:00,"['Środkowa sesja tygodnia zakończyła się spadkami głównych indeksów na GPW, co kontrastowało ze wzrostami na najważniejszych parkietach europejskich rynków bazowych. Mocno ciążyła słabość złotego, a echem odbijały się szczegóły projektu budżetu państwa.\xa0']"
4,https://www.bankier.pl/wiadomosc/Ostatnia-godzina-zadecydowala-o-wynikach-sesji-na-GPW-WIG20-na-plusie-8802920.html?utm_source=RSS&utm_medium=RSS&utm_campaign=Wiadomosci,2024-08-27 17:34:00,"['Nerwowa sesja na GPW zakończyła się ostatecznie zielonym kolorem dla WIG20, mWIG40 i WIG, dzięki mocnej ostatniej godzinie handlu. Głównym rozdającym były banki, których przecena ciążyła przez większość sesji głównym indeksom. Odrabianie strat w końcówce pozwoliło jednak przy wzrostach innych sektorów zakończyć dzień na plusie.']"
5,https://www.bankier.pl/wiadomosc/Wyniki-spolek-miliarderow-dane-z-polskiej-gospodarki-i-amerykanski-rozgrywajacy-8801140.html?utm_source=RSS&utm_medium=RSS&utm_campaign=Wiadomosci,2024-08-23 15:45:00,"['Mijający tydzień obfitował w dane makro z polskiej gospodarki, ale też wyniki największych spółek z WIG20. Tempo inwestycyjnej gry nadawane było jednak przez Amerykanów, oczekujących, jak reszta rynków finansowych, na przekaz płynący z dorocznej konferencji bankierów centralnych z Jackson Hole. W podsumowaniu giełdowego tygodnia rozmawiamy z Sobiesławem Kozłowskim z Noble Securities.']"


In [22]:
articles.shape

(247, 3)

In [23]:
def regex(clean):
    clean = re.sub(r'[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ\s]', '', clean) #removes non-letters, polish characters and space
    clean = clean.lower() #converts every letter to lowercase
    return clean

In [24]:
def tokenizer(clean):
    tokens =  word_tokenize(clean, language='polish') #tokenizes cleaned text into individual words using polish settings
    return ' '.join(tokens) #joins tokens together with spaces inbetween

In [25]:
stop_words = stopwords.stopwords('pl') #defines stopwords in polish

In [26]:
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words] #returns tokens with stopwords removed

In [27]:
#converts every token in text to its base-root
def lemmatizer(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return lemmas 

In [28]:
#applying all the above operations to text and creating final articles

cleaned_articles = articles['text'].apply(regex) #cleaning
tokenized_articles = cleaned_articles.apply(tokenizer) #tokenization
lemmatized_articles = tokenized_articles.apply(lemmatizer) #lemmatization
final_articles = lemmatized_articles.apply(remove_stop_words) #stopwords removal

In [29]:
pl_dict = pd.DataFrame(pd.read_csv('slownikWydzwieku01.csv', sep='\t', names=['word', 
                                                                              'class', 
                                                                              '2_fold', 
                                                                              '3_fold', 
                                                                              '5_fold', 
                                                                              'SO_PMI'])) #reading polish dictionary and naming columns
pl_dict.head()

,word,class,2_fold,3_fold,5_fold,SO_PMI
0,poddany,c,0,-1,-1,-4.6
1,przewrażliwiony,m,1,1,-2,-3.6
2,cudzoziemiec,c,0,0,0,-2.6
3,przekonywać,m,1,1,0,-1.8
4,skrupuł,c,1,1,1,-3.0


In [30]:
def sentiment_analysis(article_words):
    
    score_2_fold = 0
    score_3_fold = 0
    score_SO_PMI = 0
    matched_words_count = 0
    
    for word in article_words:
        match = pl_dict[pl_dict['word'] == word]
        if not match.empty:
            score_2_fold += match.iloc[0]['2_fold']
            score_3_fold += match.iloc[0]['3_fold']
            score_SO_PMI += match.iloc[0]['SO_PMI']
            matched_words_count += 1
    
    if matched_words_count > 0:        
        normalized_score_2_fold = score_2_fold / matched_words_count
        normalized_score_3_fold = score_3_fold / matched_words_count
        normalized_score_SO_PMI = score_SO_PMI / matched_words_count
    else:
        normalized_score_2_fold = 0
        normalized_score_3_fold = 0
        normalized_score_SO_PMI = 0
    
    return normalized_score_2_fold, normalized_score_3_fold, normalized_score_SO_PMI

In [31]:
articles['processed_text'] = final_articles
sentiment_scores = articles['processed_text'].apply(sentiment_analysis)
articles[['sentiment_score_2_fold', 
          'sentiment_score_3_fold', 
          'sentiment_score_SO_PMI']] = pd.DataFrame(sentiment_scores.tolist(), index=articles.index)

In [32]:
articles.loc[articles['sentiment_score_2_fold'] < 0.3, 'sentiment_score_2_fold'] = 0 #negative
articles.loc[articles['sentiment_score_2_fold'] >= 0.3, 'sentiment_score_2_fold'] = 1 #positive

articles.loc[articles['sentiment_score_3_fold'] < 0.2, 'sentiment_score_3_fold'] = -1 #negative
articles.loc[(articles['sentiment_score_3_fold'] >= 0.2) & (articles['sentiment_score_3_fold'] < 0.4), 'sentiment_score_3_fold'] = 0 #neutral
articles.loc[articles['sentiment_score_3_fold'] >= 0.4, 'sentiment_score_3_fold'] = 1 #positive 

In [33]:
sentiment = articles[['date', 
                      'text', 
                      'sentiment_score_2_fold', 
                      'sentiment_score_3_fold', 
                      'sentiment_score_SO_PMI']]

sentiment = sentiment.round({'sentiment_score_SO_PMI': 2})
sentiment['date'] = sentiment['date'].dt.date
sentiment.head()

,date,text,sentiment_score_2_fold,sentiment_score_3_fold,sentiment_score_SO_PMI
1,2024-08-30,"['Ostatnia sesja miesiąca potwierdziła silne przywiązanie WIG20 do poziomu w okolicach 2400 pkt. W efekcie od połowy sierpnia obserwujemy trend boczny, z którego coraz trudniej się wyrwać. Portfelowe przetasowania zdecydowały o wyniku sesji i ogromnych obrotach.']",1.0,0.0,1.98
0,2024-08-30,"['Ostatni tydzień wakacji na rynkach finansowych był podporządkowany wynikom Nvidii. Inwestorzy na krajowym parkiecie dostali natomiast raport CD Projektu. Kurs największego polskiego studia gier jest w tym roku najlepszy w portfelu spółek z WIG20, ale jego cena akcji zdaje się być coraz bardziej wymagająca. Nie ominiemy także podsumowania rywalizacji w grze giełdowej „Wakacje na giełdzie”.']",1.0,0.0,1.99
2,2024-08-28,"['Środkowa sesja tygodnia zakończyła się spadkami głównych indeksów na GPW, co kontrastowało ze wzrostami na najważniejszych parkietach europejskich rynków bazowych. Mocno ciążyła słabość złotego, a echem odbijały się szczegóły projektu budżetu państwa.\xa0']",1.0,0.0,2.73
4,2024-08-27,"['Nerwowa sesja na GPW zakończyła się ostatecznie zielonym kolorem dla WIG20, mWIG40 i WIG, dzięki mocnej ostatniej godzinie handlu. Głównym rozdającym były banki, których przecena ciążyła przez większość sesji głównym indeksom. Odrabianie strat w końcówce pozwoliło jednak przy wzrostach innych sektorów zakończyć dzień na plusie.']",0.0,-1.0,0.74
5,2024-08-23,"['Mijający tydzień obfitował w dane makro z polskiej gospodarki, ale też wyniki największych spółek z WIG20. Tempo inwestycyjnej gry nadawane było jednak przez Amerykanów, oczekujących, jak reszta rynków finansowych, na przekaz płynący z dorocznej konferencji bankierów centralnych z Jackson Hole. W podsumowaniu giełdowego tygodnia rozmawiamy z Sobiesławem Kozłowskim z Noble Securities.']",1.0,0.0,1.98


In [34]:
sentiment.to_csv('sentiment.csv')